# Visualizing The Sensitivity

## Interactive Experiment Comparison Tool

This notebook provides an interactive visualizer for comparing experiments across different YAML result files.

### Features:
- **Select Experiments**: Choose any two experiments from any result files using dropdown menus
- **Compare Properties**: Select which property to compare between the two experiments
- **Automatic Visualization**:
  - **Lists**: Plotted with overlay and difference views, including statistics
  - **Dictionaries**: Displayed in a formatted side-by-side table
  - **Scalars**: Shown with computed differences and relative changes

### Usage:
1. Run all cells below
2. Use the dropdowns to select:
   - A YAML file
   - Two experiments from that file to compare
   - Property to compare
3. The visualization updates automatically when you change selections

In [1]:
import yaml
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, Dropdown, VBox, HBox, Output, HTML
from IPython.display import display, clear_output
import os

In [2]:
# Load YAML files from results directory
def load_yaml_files(directory='../results/'):
    """Load all YAML files from the specified directory"""
    yaml_files = {}
    for filename in os.listdir(directory):
        if filename.endswith('.yaml'):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r') as f:
                yaml_files[filename] = yaml.safe_load(f)
    return yaml_files

# Load the data
yaml_data = load_yaml_files()
print(f"Loaded {len(yaml_data)} YAML files:")
for filename in yaml_data.keys():
    print(f"  - {filename}")

Loaded 6 YAML files:
  - overlap_comparison_results.yaml
  - false_fetal_f_results.yaml
  - detector_comparison_results.yaml
  - sensitivity_comparison_results.yaml
  - overlap_comparison_results2.yaml
  - sensitivity_comparison_results2.yaml


In [ ]:
class ExperimentComparator:
    """Interactive experiment comparison visualizer"""
    
    def __init__(self, yaml_data):
        self.yaml_data = yaml_data
        self.file_dropdown = None
        self.exp_dropdown_1 = None
        self.exp_dropdown_2 = None
        self.property_dropdown = None
        self.output = Output()
        
    def get_experiments(self, filename):
        """Get list of experiments from a YAML file"""
        if filename in self.yaml_data:
            return list(self.yaml_data[filename].keys())
        return []
    
    def get_properties(self, filename, exp_name):
        """Get list of properties from an experiment"""
        if filename in self.yaml_data and exp_name in self.yaml_data[filename]:
            return list(self.yaml_data[filename][exp_name].keys())
        return []
    
    def get_all_common_properties(self):
        """Get properties that exist in at least some experiments"""
        all_props = set()
        for filename in self.yaml_data:
            for exp in self.yaml_data[filename]:
                all_props.update(self.yaml_data[filename][exp].keys())
        return sorted(list(all_props))
    
    def compare_property(self, filename, exp1, exp2, property_name):
        """Compare a specific property between two experiments"""
        with self.output:
            clear_output(wait=True)
            
            # Get data
            data1 = None
            data2 = None
            
            if filename in self.yaml_data and exp1 in self.yaml_data[filename]:
                data1 = self.yaml_data[filename][exp1].get(property_name, None)
            
            if filename in self.yaml_data and exp2 in self.yaml_data[filename]:
                data2 = self.yaml_data[filename][exp2].get(property_name, None)
            
            # Display comparison
            print(f"\\n{'='*80}")
            print(f"Comparing: {property_name}")
            print(f"{'='*80}\\n")
            
            print(f"File: {filename}")
            print(f"Experiment 1: {exp1}")
            print(f"Experiment 2: {exp2}")
            print(f"\\n{'-'*80}\\n")
            
            # Check if both have the property
            if data1 is None and data2 is None:
                print(f"⚠ Property '{property_name}' not found in either experiment")
                return
            elif data1 is None:
                print(f"⚠ Property '{property_name}' not found in Experiment 1")
                print(f"\\nExperiment 2 value:")
                self._display_value(data2, "Exp 2")
                return
            elif data2 is None:
                print(f"⚠ Property '{property_name}' not found in Experiment 2")
                print(f"\\nExperiment 1 value:")
                self._display_value(data1, "Exp 1")
                return
            
            # Both have the property - compare them
            if isinstance(data1, list) and isinstance(data2, list):
                self._plot_lists(data1, data2, property_name, exp1, exp2)
            elif isinstance(data1, dict) and isinstance(data2, dict):
                self._display_dicts(data1, data2, exp1, exp2)
            else:
                self._display_scalars(data1, data2, exp1, exp2)
    
    def _display_value(self, value, label):
        """Display a single value"""
        if isinstance(value, list):
            plt.figure(figsize=(10, 4))
            plt.plot(value, marker='o', linestyle='-', linewidth=2, markersize=4)
            plt.title(f"{label}")
            plt.xlabel("Index")
            plt.ylabel("Value")
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
        elif isinstance(value, dict):
            print(f"\\n{label} (Dictionary):")
            for k, v in value.items():
                print(f"  {k}: {v}")
        else:
            print(f"\\n{label}: {value}")
    
    def _plot_lists(self, list1, list2, property_name, exp1, exp2):
        """Plot two lists for comparison"""
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Plot 1: Overlay
        axes[0].plot(list1, label=f"{exp1}", marker='o', linestyle='-', linewidth=2, markersize=4, alpha=0.7)
        axes[0].plot(list2, label=f"{exp2}", marker='s', linestyle='--', linewidth=2, markersize=4, alpha=0.7)
        axes[0].set_title(f"{property_name} - Overlay")
        axes[0].set_xlabel("Index")
        axes[0].set_ylabel("Value")
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Plot 2: Difference
        min_len = min(len(list1), len(list2))
        if min_len > 0:
            diff = [list1[i] - list2[i] for i in range(min_len)]
            axes[1].plot(diff, marker='o', linestyle='-', linewidth=2, markersize=4, color='red')
            axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
            axes[1].set_title(f"{property_name} - Difference (Exp1 - Exp2)")
            axes[1].set_xlabel("Index")
            axes[1].set_ylabel("Difference")
            axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Print statistics
        print(f"\\nStatistics:")
        print(f"  Length - Exp 1: {len(list1)}, Exp 2: {len(list2)}")
        if list1:
            print(f"  Exp 1 - Min: {min(list1):.6e}, Max: {max(list1):.6e}, Mean: {np.mean(list1):.6e}")
        if list2:
            print(f"  Exp 2 - Min: {min(list2):.6e}, Max: {max(list2):.6e}, Mean: {np.mean(list2):.6e}")
        if min_len > 0:
            print(f"  Difference - Min: {min(diff):.6e}, Max: {max(diff):.6e}, Mean: {np.mean(diff):.6e}")
    
    def _display_dicts(self, dict1, dict2, exp1, exp2):
        """Display two dictionaries side by side"""
        all_keys = sorted(set(list(dict1.keys()) + list(dict2.keys())))
        
        print(f"\\n{'Key':<40} {'Exp 1':<30} {'Exp 2':<30}")
        print("-" * 100)
        
        for key in all_keys:
            val1 = dict1.get(key, "N/A")
            val2 = dict2.get(key, "N/A")
            
            # Format values
            val1_str = f"{val1:.6e}" if isinstance(val1, (int, float)) and val1 != "N/A" else str(val1)
            val2_str = f"{val2:.6e}" if isinstance(val2, (int, float)) and val2 != "N/A" else str(val2)
            
            print(f"{key:<40} {val1_str:<30} {val2_str:<30}")
    
    def _display_scalars(self, val1, val2, exp1, exp2):
        """Display scalar values"""
        print(f"\\nExperiment 1 ({exp1}): {val1}")
        print(f"Experiment 2 ({exp2}): {val2}")
        
        # Try to compute difference if both are numeric
        try:
            if isinstance(val1, (int, float)) and isinstance(val2, (int, float)):
                diff = val1 - val2
                rel_diff = (diff / val1 * 100) if val1 != 0 else float('inf')
                print(f"\\nDifference (Exp1 - Exp2): {diff}")
                print(f"Relative Difference: {rel_diff:.2f}%")
        except:
            pass
    
    def create_ui(self):
        """Create the interactive UI"""
        # Single file dropdown
        file_list = sorted(list(self.yaml_data.keys()))
        self.file_dropdown = Dropdown(options=file_list, description='File:', style={'description_width': '100px'})
        
        # Experiment dropdowns
        self.exp_dropdown_1 = Dropdown(options=[], description='Exp 1:', style={'description_width': '100px'})
        self.exp_dropdown_2 = Dropdown(options=[], description='Exp 2:', style={'description_width': '100px'})
        
        # Property dropdown
        self.property_dropdown = Dropdown(options=self.get_all_common_properties(), 
                                         description='Property:', 
                                         style={'description_width': '100px'})
        
        # Update experiment dropdowns when file changes
        def update_experiments(change):
            exps = self.get_experiments(change['new'])
            self.exp_dropdown_1.options = exps
            self.exp_dropdown_2.options = exps
            if exps:
                self.exp_dropdown_1.value = exps[0]
                if len(exps) > 1:
                    self.exp_dropdown_2.value = exps[1]
                else:
                    self.exp_dropdown_2.value = exps[0]
        
        self.file_dropdown.observe(update_experiments, names='value')
        
        # Initialize experiment dropdowns
        if file_list:
            update_experiments({'new': file_list[0]})
        
        # Update comparison when any dropdown changes
        def update_comparison(*args):
            if (self.file_dropdown.value and self.exp_dropdown_1.value and 
                self.exp_dropdown_2.value and self.property_dropdown.value):
                self.compare_property(
                    self.file_dropdown.value,
                    self.exp_dropdown_1.value,
                    self.exp_dropdown_2.value,
                    self.property_dropdown.value
                )
        
        self.file_dropdown.observe(update_comparison, names='value')
        self.exp_dropdown_1.observe(update_comparison, names='value')
        self.exp_dropdown_2.observe(update_comparison, names='value')
        self.property_dropdown.observe(update_comparison, names='value')
        
        # Layout
        title = HTML("<h2>Experiment Comparison Visualizer</h2>")
        
        file_box = VBox([
            HTML("<h3>Select File</h3>"),
            self.file_dropdown
        ])
        
        exp_box = VBox([
            HTML("<h3>Select Experiments to Compare</h3>"),
            self.exp_dropdown_1,
            self.exp_dropdown_2
        ])
        
        property_box = VBox([HTML("<h3>Select Property to Compare</h3>"), self.property_dropdown])
        
        ui = VBox([title, file_box, exp_box, property_box, self.output])
        
        display(ui)
        
        # Trigger initial comparison
        update_comparison()

# Create and display the comparator
comparator = ExperimentComparator(yaml_data)
comparator.create_ui()